In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import matplotlib.cm as cm

# Vérifier la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [159]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [160]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image

In [ ]:
pip install grad-cam

In [ ]:
pip install opencv-contrib-python

In [163]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

# Define SENet Module
class SENet(nn.Module):
    def _init_(self, channels, reduction=16):
        super(SENet, self)._init_()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

# Define CBAM Module
class CBAMBlock(nn.Module):
    def _init_(self, channels, reduction=16, kernel_size=7):
        super(CBAMBlock, self)._init_()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.channel_attention(x)
        x = x * y
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.spatial_attention(y)
        x = x * y
        return x

# Define Hybrid Module (SENet + CBAM)
class Hybrid(nn.Module):
    def _init_(self, channels):
        super(Hybrid, self)._init_()
        self.senet = SENet(channels)
        self.cbam = CBAMBlock(channels)

    def forward(self, x):
        x = self.senet(x)
        x = self.cbam(x)
        return x

# Apply Attention Modules to SqueezeNet
def modify_model_with_attention(model, attention_module, layer_name):
    for name, module in model.named_modules():
        if name == layer_name:
            out_channels = module.out_channels if isinstance(module, nn.Conv2d) else None
            if out_channels:
                if attention_module == 'SENet':
                    attention = SENet(out_channels)
                elif attention_module == 'CBAM':
                    attention = CBAMBlock(out_channels)
                elif attention_module == 'Hybrid':
                    attention = Hybrid(out_channels)
                else:
                    continue
                setattr(model, name, nn.Sequential(module, attention))
    return model

# Data Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training Function
def train_model(model, train_loader, test_loader, epochs, criterion, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_loss, val_loss, train_acc, val_acc = [], [], [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss, correct_train = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            correct_train += (outputs.argmax(1) == labels).sum().item()

        model.eval()
        total_val_loss, correct_val = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                correct_val += (outputs.argmax(1) == labels).sum().item()

        train_loss.append(total_train_loss / len(train_loader))
        val_loss.append(total_val_loss / len(test_loader))
        train_acc.append(correct_train / len(train_loader.dataset))
        val_acc.append(correct_val / len(test_loader.dataset))

        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return train_loss, val_loss, train_acc, val_acc

# Model Definitions
squeezenet_base = models.squeezenet1_0(pretrained=True)
squeezenet_senet = modify_model_with_attention(models.squeezenet1_0(pretrained=True), 'SENet', 'features.12')
squeezenet_cbam = modify_model_with_attention(models.squeezenet1_0(pretrained=True), 'CBAM', 'features.12')
squeezenet_hybrid = modify_model_with_attention(models.squeezenet1_0(pretrained=True), 'Hybrid', 'features.12')

models_to_train = {
    "SqueezeNet Base": squeezenet_base,
    "SqueezeNet + SENet": squeezenet_senet,
    "SqueezeNet + CBAM": squeezenet_cbam,
    "SqueezeNet + Hybrid": squeezenet_hybrid
}

criterion = nn.CrossEntropyLoss()
learning_rate = 0.003
epochs = 50
results = {}

for name, model in models_to_train.items():
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Training {name}")
    results[name] = train_model(model, train_loader, test_loader, epochs, criterion, optimizer)

# Plotting Results
plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_loss, label=f'{name} Train Loss')
    plt.plot(val_loss, label=f'{name} Val Loss')
plt.title('Loss Curves')
plt.legend()
plt.show()

plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_acc, label=f'{name} Train Accuracy')
    plt.plot(val_acc, label=f'{name} Val Accuracy')
plt.title('Accuracy Curves')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# Define your DataLoader objects for training and validation
dataloaders = {
    'train': train_loader,  # Replace with your actual train DataLoader
    'val': test_loader  # Replace with your actual validation DataLoader
}

# Define the class names for CIFAR-10
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# Define the imshow function (if not already defined)
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)  # Ensure the data is in the range [0, 1] for display
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Visualize model predictions
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=(15, 6))

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(2, num_images // 2, images_so_far)  # Arrange images in 2 rows
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
    model.train(mode=was_training)

# Call the visualization function
visualize_model(model)

plt.ioff()
plt.show()

In [ ]:
def print_prediction_summary(preds, class_names):
    prediction_counts = {class_name: 0 for class_name in class_names}
    for pred in preds:
        prediction_counts[class_names[pred]] += 1
    print("Prediction Summary:")
    for class_name, count in prediction_counts.items():
        print(f"{class_name}: {count}")

# Visualize model predictions with additional debug information
def visualize_model_grid_debug(model, num_classes=10, samples_per_class=5):
    was_training = model.training
    model.eval()
    fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(15, 15))

    counters = [0] * num_classes
    all_preds = []

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())

            for j in range(inputs.size()[0]):
                class_idx = preds[j].item()

                if counters[class_idx] < samples_per_class:
                    ax = axes[class_idx, counters[class_idx]]
                    ax.axis('off')

                    img = inputs.cpu().data[j].numpy().transpose(1, 2, 0)
                    img = (img - img.min()) / (img.max() - img.min())  # Normalize
                    ax.imshow(img)
                    ax.set_title(f'Predicted: {class_names[class_idx]}')

                    counters[class_idx] += 1

                if sum(counters) >= num_classes * samples_per_class:
                    model.train(mode=was_training)
                    plt.tight_layout()
                    plt.show()
                    print_prediction_summary(all_preds, class_names)
                    return

    model.train(mode=was_training)
    plt.tight_layout()
    plt.show()
    print_prediction_summary(all_preds, class_names)

# Call the visualization function with debugging
visualize_model_grid_debug(model)

plt.ioff()
plt.show()

Resnet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

# Define SENet Module
class SENet(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SENet, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

# Define CBAM Module
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.channel_attention(x)
        x = x * y
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.spatial_attention(y)
        x = x * y
        return x

# Define Hybrid Module (SENet + CBAM)
class Hybrid(nn.Module):
    def __init__(self, channels):
        super(Hybrid, self).__init__()
        self.senet = SENet(channels)
        self.cbam = CBAMBlock(channels)

    def forward(self, x):
        x = self.senet(x)
        x = self.cbam(x)
        return x

# Apply Attention Modules to ResNet18
def modify_model_with_attention(model, attention_module, layer_name):
    for name, module in model.named_modules():
        if name == layer_name:
            if isinstance(module, nn.Conv2d):
                out_channels = module.out_channels
            else:
                continue  # Ignorer les couches non Conv2d

            if attention_module == 'SENet':
                attention = SENet(out_channels)
            elif attention_module == 'CBAM':
                attention = CBAMBlock(out_channels)
            elif attention_module == 'Hybrid':
                attention = Hybrid(out_channels)
            else:
                raise ValueError("Module d'attention non reconnu")

            # Remplacer la couche par une séquence (couche originale + module d'attention)
            parent_name = '.'.join(layer_name.split('.')[:-1])
            child_name = layer_name.split('.')[-1]
            parent_module = model
            for part in parent_name.split('.'):
                parent_module = getattr(parent_module, part)
            setattr(parent_module, child_name, nn.Sequential(module, attention))

    return model

# Data Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training Function
def train_model(model, train_loader, test_loader, epochs, criterion, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_loss, val_loss, train_acc, val_acc = [], [], [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss, correct_train = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            correct_train += (outputs.argmax(1) == labels).sum().item()

        model.eval()
        total_val_loss, correct_val = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                correct_val += (outputs.argmax(1) == labels).sum().item()

        train_loss.append(total_train_loss / len(train_loader))
        val_loss.append(total_val_loss / len(test_loader))
        train_acc.append(correct_train / len(train_loader.dataset))
        val_acc.append(correct_val / len(test_loader.dataset))

        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return train_loss, val_loss, train_acc, val_acc

# Model Definitions
resnet_base = models.resnet18(pretrained=True)
resnet_senet = modify_model_with_attention(models.resnet18(pretrained=True), 'SENet', 'layer4.1.conv2')
resnet_cbam = modify_model_with_attention(models.resnet18(pretrained=True), 'CBAM', 'layer4.1.conv2')
resnet_hybrid = modify_model_with_attention(models.resnet18(pretrained=True), 'Hybrid', 'layer4.1.conv2')

models_to_train = {
    "ResNet Base": resnet_base,
    "ResNet + SENet": resnet_senet,
    "ResNet + CBAM": resnet_cbam,
    "ResNet + Hybrid": resnet_hybrid
}

criterion = nn.CrossEntropyLoss()
learning_rate = 0.003
epochs = 50
results = {}

for name, model in models_to_train.items():
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Training {name}")
    results[name] = train_model(model, train_loader, test_loader, epochs, criterion, optimizer)

# Plotting Results
plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_loss, label=f'{name} Train Loss')
    plt.plot(val_loss, label=f'{name} Val Loss')
plt.title('Loss Curves')
plt.legend()
plt.show()

plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_acc, label=f'{name} Train Accuracy')
    plt.plot(val_acc, label=f'{name} Val Accuracy')
plt.title('Accuracy Curves')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

# Define SENet Module
class SENet(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SENet, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

# Define CBAM Module
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.channel_attention(x)
        x = x * y
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.spatial_attention(y)
        x = x * y
        return x

# Define Hybrid Module (SENet + CBAM)
class Hybrid(nn.Module):
    def __init__(self, channels):
        super(Hybrid, self).__init__()
        self.senet = SENet(channels)
        self.cbam = CBAMBlock(channels)

    def forward(self, x):
        x = self.senet(x)
        x = self.cbam(x)
        return x

# Apply Attention Modules to VGG
def modify_model_with_attention(model, attention_module, layer_name):
    for name, module in model.named_modules():
        if name == layer_name:
            if isinstance(module, nn.Conv2d):
                out_channels = module.out_channels
            else:
                continue  # Ignorer les couches non Conv2d

            if attention_module == 'SENet':
                attention = SENet(out_channels)
            elif attention_module == 'CBAM':
                attention = CBAMBlock(out_channels)
            elif attention_module == 'Hybrid':
                attention = Hybrid(out_channels)
            else:
                raise ValueError("Module d'attention non reconnu")

            # Remplacer la couche par une séquence (couche originale + module d'attention)
            parent_name = '.'.join(layer_name.split('.')[:-1])
            child_name = layer_name.split('.')[-1]
            parent_module = model
            for part in parent_name.split('.'):
                parent_module = getattr(parent_module, part)
            setattr(parent_module, child_name, nn.Sequential(module, attention))

    return model

# Data Preparation
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Réduire la taille des images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Augmenter le batch_size
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Training Function
def train_model(model, train_loader, test_loader, epochs, criterion, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_loss, val_loss, train_acc, val_acc = [], [], [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss, correct_train = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            correct_train += (outputs.argmax(1) == labels).sum().item()

        model.eval()
        total_val_loss, correct_val = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                correct_val += (outputs.argmax(1) == labels).sum().item()

        train_loss.append(total_train_loss / len(train_loader))
        val_loss.append(total_val_loss / len(test_loader))
        train_acc.append(correct_train / len(train_loader.dataset))
        val_acc.append(correct_val / len(test_loader.dataset))

        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {train_loss[-1]:.4f}, Val Loss: {val_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return train_loss, val_loss, train_acc, val_acc

# Model Definitions
vgg_base = models.vgg16(pretrained=True)
vgg_senet = modify_model_with_attention(models.vgg16(pretrained=True), 'SENet', 'features.28')
vgg_cbam = modify_model_with_attention(models.vgg16(pretrained=True), 'CBAM', 'features.28')
vgg_hybrid = modify_model_with_attention(models.vgg16(pretrained=True), 'Hybrid', 'features.28')

models_to_train = {
    "VGG Base": vgg_base,
    "VGG + SENet": vgg_senet,
    "VGG + CBAM": vgg_cbam,
    "VGG + Hybrid": vgg_hybrid
}

criterion = nn.CrossEntropyLoss()
learning_rate = 0.003
epochs = 50
results = {}

for name, model in models_to_train.items():
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Training {name}")
    results[name] = train_model(model, train_loader, test_loader, epochs, criterion, optimizer)

# Plotting Results
plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_loss, label=f'{name} Train Loss')
    plt.plot(val_loss, label=f'{name} Val Loss')
plt.title('Loss Curves')
plt.legend()
plt.show()

plt.figure(figsize=(8, 4))
for name, (train_loss, val_loss, train_acc, val_acc) in results.items():
    plt.plot(train_acc, label=f'{name} Train Accuracy')
    plt.plot(val_acc, label=f'{name} Val Accuracy')
plt.title('Accuracy Curves')
plt.legend()
plt.show()